In [ ]:
import os
import glob
import sys

import numpy as np
import pandas as pd
from scipy.misc import derivative
import scipy.integrate as intg
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import Constant, RandomNormal

from sklearn.metrics import mean_squared_error
from astroNN.nn.layers import MCDropout

sys.path.append("/home/isidro/Documents/github/nnogada/")
from nnogada import Nnogada

np.random.seed(0)
%matplotlib inline

In [ ]:
# absolute path to search all text files inside a specific folder
path = r'/home/isidro/Documents/github/model_independent_RC/data/MassModels/*.NFW.fix.REV.dat'
files = glob.glob(path)
print(files)

In [ ]:
for file in files:
    print(file)

len(files)

In [ ]:
def model_regression_dropout(num_nodes, num_hidden_layers):
    # Defeine Keras model for regression
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(batch_input_shape=((None, 1))))
    for _ in range(num_hidden_layers):
        model.add(Dense(units=num_nodes, activation='relu'))
        model.add(MCDropout(0.2))
    model.add(Dense(units=2, activation="linear"))
    return model

In [ ]:
files[0]

In [ ]:
history_all = []

for idx, file in enumerate(files):
    print("Model {}/{}".format(idx+1, len(files)))
    print("-"*10)
    
    history_ind = {}
    
    data = np.loadtxt(files[idx], skiprows=12) 
    df = pd.DataFrame(data, columns=['Radius', 'vgas', 'vdisk', 'vbulge', 'vobs', 'err_vobs', 
                                     'Vu', 'Vt','Rxv', 'Vxy'])
    N = len(df.values)
    print("Len: ", N)
    print(df.head())
    randomize = np.random.permutation(N)
    data = df.values[randomize]

    x = data[:,0]
    y = data[:,1:3]
    
    scalerx = StandardScaler()
    scalerx.fit(x.reshape(-1,1))
    # apply transform
    x = scalerx.transform(x.reshape(-1,1))    
    split = 0.8
    ntrain = int(split * len(x))
    indx = [ntrain]
    x_train, x_test = np.split(x, indx)
    y_train, y_test = np.split(y, indx)
    print("X_train shape: {} | y_train shape: {} | x_test shape: {} | y_test shape: {}".format(np.shape(x_train), 
                                                                                               np.shape(y_train), 
                                                                                               np.shape(x_test), 
                                                                                               np.shape(y_test)))

    population_size = 4  # max of individuals per generation
    max_generations = 10    # number of generations
    gene_length = 8        # lenght of the gene, depends on how many hiperparameters are tested 2*hyp
    k = 1                  # num. of finalist individuals

    # Define the hyperparameters for the search
    hyperparams = {'deep': [3, 4], 'num_units': [100, 200], 'batch_size': [8, 16]}

    # generate a Nnogada instance
    net_fit = Nnogada(hyp_to_find=hyperparams, X_train=x_train, Y_train=y_train, X_val=x_test, Y_val=y_test, 
                      neural_library='keras', regression=True)
    # Set the possible values of hyperparameters and not use the default values from hyperparameters.py
    net_fit.set_hyperparameters()
          
    best_population = net_fit.ga_with_elitism(population_size, max_generations, gene_length, k)
          
    deep = int(best_population['deep'])
    num_units = int(best_population['num_units'])
    batch_size = int(best_population['batch_size'])
            
    # optimizer = Adam(lr=.005)
    optimizer = Adam(lr=0.0005)

    # Compile Keras model
    model = model_regression_dropout(num_nodes=num_units, num_hidden_layers=deep)
#     print(model.summary())
          
    model.compile(loss='mse', optimizer=optimizer) 
  
    model_train = model.fit(x_train, y_train, batch_size=batch_size,
                            epochs=1000, verbose=0,
                            validation_data=(x_test, y_test))
    
#     model_train.history['val_loss'][-1]
                  
    history_ind['idx'] = idx
    history_ind['deep'] = deep
    history_ind['batch_size'] = batch_size
    history_ind['num_units'] = num_units
    history_ind['loss'] = model_train.history['val_loss'][-1]
          
    history_all.append(history_ind)   
         
    model.save('models/RC_model_{}.h5'.format(idx))
  
    # Generate test data
    test_batch_size = 1000
    # x_test = np.random.uniform(0, 2., test_batch_size)
    x_test_to_pred = np.linspace(min(df['Radius'].values)-0.1, max(df['Radius'].values)+0.1, test_batch_size)

    mc_dropout_num = 100  # Run Dropout 100 times
    predictions = np.zeros((mc_dropout_num, test_batch_size, 2))
    uncertainty = np.zeros((mc_dropout_num, test_batch_size, 1))
    for i in range(mc_dropout_num):
        predictions[i] = model.predict(scalerx.transform(x_test_to_pred.reshape(-1,1)))

    # get mean results and its varience
    prediction_mc_dropout = np.mean(predictions, axis=0)
    std_mc_dropout = np.std(predictions, axis=0)
    
    # plt.rcParams["figure.figsize"] = [7.50, 3.50]
    plt.rcParams["figure.autolayout"] = True
    overlapping = 0.6

    # Plotting
    plt.figure(figsize=(10, 7), dpi=100)
    sigma = np.sqrt(std_mc_dropout[:, 0]**2 + std_mc_dropout[:, 1]**2+ prediction_mc_dropout[:,1]**2)

    plt.plot(df['Radius'].values, df['Vt'].values, color='k', linewidth=3, label='NFW theory',alpha=1)

    plt.errorbar(df['Radius'].values, df['vobs'].values, yerr=df['err_vobs'].values, fmt='.', 
                 color='red', elinewidth=1, ecolor='red', markersize=5, label='Observations', alpha=overlapping)

    plt.errorbar(x_test_to_pred, prediction_mc_dropout[:,0], yerr=sigma, markersize=2, fmt='o', 
                 ecolor='green', capthick=2, elinewidth=0.5, alpha=overlapping-0.2, c='green',
                 label='Neural reconstruction')


    plt.ylabel("$V(r)$", fontsize=20)
    plt.xlabel("r", fontsize=20)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

    plt.legend(loc='upper left')
    plt.tight_layout()

    plt.savefig("neural_reconstruction_RC_{}.png".format(idx), dpi=100)
          
    plt.figure(figsize=(8, 6), dpi=100)

    plt.plot(model_train.history['loss'], color='r', )
    plt.plot(model_train.history['val_loss'], color='g')

    plt.ylabel('MSE', fontsize=11)
    plt.xlabel('Epoch', fontsize=11)
    plt.legend(['training set', 'validation set'], loc='upper right', fontsize=12)
    plt.xticks(fontsize=10)
    plt.savefig('loss_fn_neural_RC_{}'.format(idx), dpi=100)
    print("-"*10)
    # plt.ylim(0, 200)

